# Graph Neural Network

## Introducing the Model

We have already laid out the basic idea of Graph Neural Networks (GNNs) in the introduction, and as such we will instead introduce a useful variant, known as Graph Convolutional Networks (GCNs). GCNs are the most prevelant and broadly applied model of GNNs, [1] which are able to leverage both the information of the nodes and their locality to make predictions [2]. They do this using a convolution layer, which merges the features of a node with those of its neighbours. The most common method of recommender systems used with GCNs is Collaborative Filtering, where user's past interactions and connections to other users is used to make predictions. The main way it does this is through learning latent features (or an embedding) for each user and item node. GCNs then update this embedding by propagating information across the graph:

Let $\mathbf{e}_u^{(0)}$ denote the user embedding of user $u$, and $\mathbf{e}_i^{(0)}$ the item embedding of item $i$. Then after $k$ layers of propagation, we have:

$$\mathbf{e}_u^{(k+1)} = \sigma \left( \mathbf{W}_1 \mathbf{e}_u^{(k)} + \sum_{i\in \mathcal{N}_u} \frac{1}{\sqrt{|\mathcal{N}_u||\mathcal{N}_i|}} (\mathbf{W}_1\mathbf{e}_u^{(k)} + \mathbf{W}_2(\mathbf{e}_i^{(k)}\odot \mathbf{e}_u^{(k)}))\right)$$

and a similar equation for $\mathbf{e}_i^{(k+1)}$ [3]. In this equation, $\sigma$ is the nonlinear activation function (e.g. ReLU), $\mathcal{N}_u$ denotes the set of items that are interacted with by user $u$, and similar for $\mathcal{N}_i$ and item $i$. $\mathbf{W}_1$ and $\mathbf{W}_2$ are trainable weight matrices used to perform feature transformations in each layer.

These equations represent the basic building block of most GCNs, however after recent investigations as published in paper [3], it was discovered that for recommendation systems in particular, both the activation function, $\sigma$, and the feature transformation matrices, $\mathbf{W}_1, \mathbf{W}_2$, have little influence on the model and in fact make the predictions worse in some situations. As such a new model was suggested that stripped these parts for a much simpler propogation method:

$$\mathbf{e}_u^{(k+1)} = \sum_{i\in \mathcal{N}_u} \frac{1}{\sqrt{|\mathcal{N}_u||\mathcal{N}_i|}} \mathbf{e}_i^{(k)}$$

$$\mathbf{e}_i^{(k+1)} = \sum_{u\in \mathcal{N}_i} \frac{1}{\sqrt{|\mathcal{N}_i||\mathcal{N}_u|}} \mathbf{e}_u^{(k)}$$

This model is known as LightGCN, and was shown to perform better in most cases than the other prevalent GCN and GNN models for recommendation systems. This goes to show that more complicated is not always better, which carries over to data at scale. Sometimes one of the best ways to deal with big data is to make sure your model isn't performing unnecessary calculations.

LightGCN is still among the best models for collaborative filtering in recommender systems, and as such is the one that we will consider here.

## Importing Modules and Reading Data

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import os
import sys
from sklearn.model_selection import train_test_split

cwd = os.getcwd()
sys.path.append(os.path.join('.','scripts'))

import GraphFuncs as gf

In [2]:
artists = pd.read_csv(os.path.join(cwd,'data','artists.dat'), delimiter='\t')
tags = pd.read_csv(os.path.join(cwd,'data','tags.dat'), delimiter='\t',encoding='ISO-8859-1')
user_artists = pd.read_csv(os.path.join(cwd,'data','user_artists.dat'), delimiter='\t')
user_friends = pd.read_csv(os.path.join(cwd,'data','user_friends.dat'), delimiter='\t')
user_taggedartists_timestamps = pd.read_csv(os.path.join(cwd,'data','user_taggedartists-timestamps.dat'), delimiter='\t')
user_taggedartists = pd.read_csv(os.path.join(cwd,'data','user_taggedartists.dat'), delimiter='\t')

## Deciding the Question

The goal of a recommender system is to try to predict which items a user will like, and then give them recommendations based on this. In this section, for our dataset, we will look at the goal of predicting whether or not a user will like an artist. To achieve this, we will create an 80-20 training-testing split of the data. We have seen that for most users, the `user_artist` dataset contains their top 50 artists, so we will take 40 of them for each user at random, and isolate the other 10 to use as testing data. We will then train a GNN on the 40 artists in the training dataset, and evaluate the model by seeing how it predicts if the user will like the 10 artists in the testing data. If the model performs well on this target, then we could use the model to predict whether or not a user will like an artist not contained in their 50 artists.

## Creating the Train-Test Split

In order to create the train-test split we will use the Scikit-learn module, which has a built in function for it. It also has a built in `stratify` parameter, which allows us to specify that the data should be split whilst keeping the proportion of data the same for each user, i.e. what we described above. However this requires that each user has atleast 2 artist data entries, but we see that this is not the case:

In [3]:
users = user_artists['userID'].unique()
singleartistusers = [int(user) for user in users if len(gf.get_artists(user,user_artists)) == 1]
print(singleartistusers)

[112, 615, 1013, 1307, 1603, 1731, 1758, 2085]


We see that there are 8 users with only 1 artist data entry. As such we remove those, create the train-test split, and then re-add those users back to the training data:

In [4]:
# Isolate the rows of data for the users which only have one artist data point.
singleartistusersdf = user_artists[user_artists['userID'].isin(singleartistusers)]
# Take the rest of the data into a temp df
user_artists_temp = user_artists[~user_artists['userID'].isin(singleartistusers)]

# Create the train-test split using this temp df
user_artists_train, user_artists_test = train_test_split(user_artists_temp,
                                                          test_size = 0.2, 
                                                          stratify = user_artists_temp['userID'], 
                                                          random_state = 47)

# Concat the dfs back together
user_artists_train = pd.concat([user_artists_train,singleartistusersdf])

Then we save the datasets as text files so that the model can access them.

In [5]:
dfs = [user_artists_train, user_artists_test, user_friends]

filepath = os.path.join(cwd,'data','GNN')
if not os.path.exists(filepath):
    os.makedirs(filepath)

for df in dfs:
    df.to_csv(os.path.join(filepath,gf.get_df_name(df, globals()) + '.txt'),sep='\t',header=False,index=False)

## Exploring the Model

In order to use the LightGCN model, we have downloaded the scripts from the GitHub repository [4] associated with the paper [3] that suggested the model. This repository already had a Last.fm dataset built in, however it is different to ours and as such we must alter the code to run to our needs. This mostly involved adding a new class in the `dataloader.py` script, known as LastFM2, which was done by altering the already existing LastFM class.

The downloaded scripts are contained in the `scripts/LightGCN` directory.

In [16]:
with open(os.path.join(cwd,'scripts','LightGCN','dataloader.py')) as file:
    content = file.read()
    start = content.index('class LastFM2')
    print(content[start:])

class LastFM2(BasicDataset):
    """
    Dataset type for pytorch
    LastFM dataset 2
    """
    def __init__(self, path=os.path.join(cwd,'data','GNN')):
        # train or test
        cprint("loading [last fm 2]")
        self.mode_dict = {'train':0, "test":1}
        self.mode    = self.mode_dict['train']
        # self.n_users = 2100
        # self.m_items = 18745
        trainData = pd.read_table(join(path, 'user_artists_train.txt'), header=None)
        print(trainData.head())
        testData  = pd.read_table(join(path, 'user_artists_test.txt'), header=None)
        print(testData.head())
        trustNet  = pd.read_table(join(path, 'user_friends.txt'), header=None).to_numpy()
        print(trustNet[:5])
        trustNet -= 1
        trainData-= 1
        testData -= 1
        self.trustNet  = trustNet
        self.trainData = trainData
        self.testData  = testData
        self.trainUser = np.array(trainData[:][0])
        self.trainUniqueUsers = np.unique(self.trainUser)

We have also made other alterations to the code to suit our needs for the model. These changes will be annotated in the scripts themself where appropriate.

# References

[1] Papers With Code: Graph Models - https://paperswithcode.com/methods/category/graph-models

[2] Towards Data Science: Graph Convolutional Networks - https://towardsdatascience.com/graph-convolutional-networks-introduction-to-gnns-24b3f60d6c95

[3] Xiangnan He, et al. 2020. LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation - https://arxiv.org/abs/2002.02126

[4] Guyse1234 Github Repository: LightGCN-PyTorch - https://github.com/gusye1234/LightGCN-PyTorch